In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('./data.csv')

In [3]:
for name in df.columns:
    print(f'- {name} -------------------------')
    print(df[name].value_counts())

- ID -------------------------
1     1
2     1
21    1
20    1
19    1
18    1
17    1
16    1
15    1
14    1
13    1
12    1
11    1
10    1
9     1
8     1
7     1
6     1
5     1
4     1
3     1
22    1
Name: ID, dtype: int64
- name -------------------------
Alice        1
Bob          1
Valentino    1
Urine        1
Tate         1
Stewart      1
Ralf         1
Paton        1
ONeill       1
Nathan       1
Marries      1
Louis        1
Kate         1
Jamie        1
Iris         1
Helen        1
Gordon       1
Fouler       1
Eli          1
Derrick      1
Charles      1
Warner       1
Name: name, dtype: int64
- SwE -------------------------
1    13
0     9
Name: SwE, dtype: int64
- PdM -------------------------
0    19
1     3
Name: PdM, dtype: int64
- Biz -------------------------
0    19
1     3
Name: Biz, dtype: int64
- CS -------------------------
0    19
1     3
Name: CS, dtype: int64


In [4]:
len_members = len(df)
len_members

22

In [5]:
len_groups = int(len_members / 3)
len_groups

7

## Requirements
- Members must be assigned to one group
- Group members must be 3 to 4
- Max members of SwE must be 2

In [53]:
import pulp
problem = pulp.LpProblem("Group", pulp.LpMaximize)

In [54]:
## Variables

In [55]:
import string
G = list(string.ascii_uppercase[:len_groups])
G

['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [56]:
M = df['ID'].tolist()
M

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [57]:
MG = [(m,g) for m in M for g in G]
pd.Series(MG)

0       (1, A)
1       (1, B)
2       (1, C)
3       (1, D)
4       (1, E)
        ...   
149    (22, C)
150    (22, D)
151    (22, E)
152    (22, F)
153    (22, G)
Length: 154, dtype: object

In [58]:
X = pulp.LpVariable.dicts('X', MG, cat=pulp.LpBinary)

## Constraints

In [59]:
# Members must be assigned to one group
for m in M:
    problem += pulp.lpSum(X[m,g] for g in G) == 1

In [60]:
# Group members must be 3 to 4
for g in G:
    problem += pulp.lpSum(X[m,g] for m in M) >= 3
    problem += pulp.lpSum(X[m,g] for m in M) <= 4

In [61]:
# Max members of SwE must be 2
M_swe = df[df['SwE'] == 1]['ID'].tolist()
M_swe

[1, 2, 3, 4, 6, 7, 9, 12, 15, 16, 18, 19, 21]

In [62]:
for g in G:
    problem += pulp.lpSum(X[m,g] for m in M_swe) <= 2

In [63]:
status = problem.solve()
print(pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/70f10899d39544128c5567399b413df6-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/70f10899d39544128c5567399b413df6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 48 COLUMNS
At line 911 RHS
At line 955 BOUNDS
At line 1111 ENDATA
Problem MODEL has 43 rows, 155 columns and 553 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 22 SOS with 154 members
Cgl0004I processed model has 36 rows, 154 columns (154 integer (154 of which binary)) and 399 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of

In [67]:
for g in G:
    print(f'- Group: {g} --------------')
    for m in M:
        print(X[m,g].value())

- Group: A --------------
0.0
0.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
- Group: B --------------
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
- Group: C --------------
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
- Group: D --------------
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
- Group: E --------------
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
- Group: F --------------
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
- Group: G --------------
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [68]:
result_df = df.copy()

In [69]:
result_df['group'] = result_df['ID'].map({ m:g for m in M for g in G if X[m,g].value() == 1})

In [70]:
result_df

,ID,name,SwE,PdM,Biz,CS,group
0,1,Alice,1,0,0,0,G
1,2,Bob,1,0,0,0,D
2,3,Charles,1,0,0,0,C
3,4,Derrick,1,0,0,0,B
4,5,Eli,0,0,0,1,A
5,6,Fouler,1,0,0,0,B
6,7,Gordon,1,0,0,0,A
7,8,Helen,0,0,1,0,A
8,9,Iris,1,0,0,0,A
9,10,Jamie,0,1,0,0,G


In [71]:
result_df.sort_values('group')

,ID,name,SwE,PdM,Biz,CS,group
4,5,Eli,0,0,0,1,A
6,7,Gordon,1,0,0,0,A
7,8,Helen,0,0,1,0,A
8,9,Iris,1,0,0,0,A
21,22,Warner,0,0,1,0,B
3,4,Derrick,1,0,0,0,B
5,6,Fouler,1,0,0,0,B
12,13,Marries,0,0,0,1,C
2,3,Charles,1,0,0,0,C
20,21,Valentino,1,0,0,0,C


In [81]:
from group import solver

p = solver.Problem('./data.csv')
result = p.run()
result.sort_values('group')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/476d901e92134365b4cc48e8eaaab61a-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/476d901e92134365b4cc48e8eaaab61a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 48 COLUMNS
At line 911 RHS
At line 955 BOUNDS
At line 1111 ENDATA
Problem MODEL has 43 rows, 155 columns and 553 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 22 SOS with 154 members
Cgl0004I processed model has 36 rows, 154 columns (154 integer (154 of which binary)) and 399 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of

,ID,name,SwE,PdM,Biz,CS,group
4,5,Eli,0,0,0,1,A
6,7,Gordon,1,0,0,0,A
7,8,Helen,0,0,1,0,A
8,9,Iris,1,0,0,0,A
21,22,Warner,0,0,1,0,B
3,4,Derrick,1,0,0,0,B
5,6,Fouler,1,0,0,0,B
12,13,Marries,0,0,0,1,C
2,3,Charles,1,0,0,0,C
20,21,Valentino,1,0,0,0,C


ランダム性が欲しい

In [178]:
result.sort_values('group').to_csv('./result_01.csv', index=False)

In [123]:
M = df['ID'].tolist()
G = list(string.ascii_uppercase[:len_groups])
MG = [(m, g) for m in M for g in G]
X = pulp.LpVariable.dicts('X', MG, cat=pulp.LpBinary)

# We want to minimize duplication
problem = pulp.LpProblem("Group", pulp.LpMinimize)

# 1. Members must be assigned to one group
for m in M:
    problem += pulp.lpSum(X[m, g] for g in G) == 1

# 2. Number of members in a group must be 3 to 4
for g in G:
    problem += pulp.lpSum(X[m, g] for m in M) >= 3
    problem += pulp.lpSum(X[m, g] for m in M) <= 4

# 3. Max members of SwE must be 2
swe = df[df['SwE'] == 1]['ID'].tolist()
for g in G:
    problem += pulp.lpSum(X[m, g] for m in swe) <= 2

In [124]:
# 4. Minimize same group assignation

prev_group = { (m, g): 0 for m in M for g in G }
for row in result.itertuples():
    prev_group[(row.ID, row.group)] = 1
problem += pulp.lpSum(X[m, g] * prev_group[m, g] for m, g in MG)

In [125]:
status = problem.solve()
print(pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/b789c9133c244db18ef4f5ebcd992b60-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/b789c9133c244db18ef4f5ebcd992b60-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 48 COLUMNS
At line 932 RHS
At line 976 BOUNDS
At line 1131 ENDATA
Problem MODEL has 43 rows, 154 columns and 553 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 22 SOS with 154 members
Cgl0004I processed model has 36 rows, 154 columns (154 integer (154 of which binary)) and 399 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers un

In [126]:
result_df = df.copy()
result_df['group'] = result_df['ID'].map({ m:g for m in M for g in G if X[m,g].value() == 1})
result_df.sort_values('group')

,ID,name,SwE,PdM,Biz,CS,group
1,2,Bob,1,0,0,0,A
19,20,Urine,0,1,0,0,A
14,15,ONeill,1,0,0,0,A
0,1,Alice,1,0,0,0,B
16,17,Ralf,0,1,0,0,B
15,16,Paton,1,0,0,0,B
17,18,Stewart,1,0,0,0,C
11,12,Louis,1,0,0,0,C
10,11,Kate,0,0,1,0,C
13,14,Nathan,0,0,0,1,D


↑ This constraint was not desirable because we want to create group with small duplication, not different group name.

In [166]:
M = df['ID'].tolist()
G = list(string.ascii_uppercase[:len_groups])
MG = [(m, g) for m in M for g in G]
X = pulp.LpVariable.dicts('X', MG, cat=pulp.LpBinary)

# We want to minimize duplication
problem = pulp.LpProblem("Group", pulp.LpMinimize)

# 1. Members must be assigned to one group
for m in M:
    problem += pulp.lpSum(X[m, g] for g in G) == 1

# 2. Number of members in a group must be 3 to 4
for g in G:
    problem += pulp.lpSum(X[m, g] for m in M) >= 3
    problem += pulp.lpSum(X[m, g] for m in M) <= 4

# 3. Max members of SwE must be 2
swe = df[df['SwE'] == 1]['ID'].tolist()
for g in G:
    problem += pulp.lpSum(X[m, g] for m in swe) <= 2

In [171]:
prev_group_mates = dict()

# for g, data in result.groupby('group'):
#     members = data['ID'].to_list()
#     for name in members:
#         prev_group_mates[name] = dict(zip(M.copy(), [0] * len(M)))
#         for m in M:
#             if len(list(filter(lambda _m: _m == m, members))) > 0:
#                 prev_group_mates[name][m] = 1

for g, data in result.groupby('group'):
    members = data['ID'].to_list()

    for name in members:
        prev_group_mates[name] = members

prev_group_mates

{5: [5, 7, 8, 9],
 7: [5, 7, 8, 9],
 8: [5, 7, 8, 9],
 9: [5, 7, 8, 9],
 4: [4, 6, 22],
 6: [4, 6, 22],
 22: [4, 6, 22],
 3: [3, 13, 21],
 13: [3, 13, 21],
 21: [3, 13, 21],
 2: [2, 19, 20],
 19: [2, 19, 20],
 20: [2, 19, 20],
 14: [14, 15, 18],
 15: [14, 15, 18],
 18: [14, 15, 18],
 12: [12, 16, 17],
 16: [12, 16, 17],
 17: [12, 16, 17],
 1: [1, 10, 11],
 10: [1, 10, 11],
 11: [1, 10, 11]}

In [172]:
for n, members in prev_group_mates.items():
    for g in G:
        problem += pulp.lpSum(X[m, g] for m in members) <= 2

In [173]:
status = problem.solve()
print(pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/a8a2a86b52d64ca58a1a679fa8e6aae5-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/a8a2a86b52d64ca58a1a679fa8e6aae5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 356 COLUMNS
At line 2199 RHS
At line 2551 BOUNDS
At line 2707 ENDATA
Problem MODEL has 351 rows, 155 columns and 1533 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 22 SOS with 154 members
Cgl0004I processed model has 85 rows, 154 columns (154 integer (154 of which binary)) and 553 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found o

In [174]:
result_df = df.copy()
result_df['group'] = result_df['ID'].map({ m:g for m in M for g in G if X[m,g].value() == 1})
result_df.sort_values('group')

,ID,name,SwE,PdM,Biz,CS,group
10,11,Kate,0,0,1,0,A
12,13,Marries,0,0,0,1,A
20,21,Valentino,1,0,0,0,A
14,15,ONeill,1,0,0,0,B
21,22,Warner,0,0,1,0,B
4,5,Eli,0,0,0,1,B
1,2,Bob,1,0,0,0,B
8,9,Iris,1,0,0,0,C
9,10,Jamie,0,1,0,0,C
11,12,Louis,1,0,0,0,C


In [177]:
result_df.sort_values('group').to_csv('./result_02.csv', index=False)

In [185]:
# from group import solver
import importlib
importlib.reload(solver)

p = solver.Problem('./data.csv', prev_group=result_df)
result3 = p.run()
result3.sort_values('group')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/fc9ea964416c483bb832ad57810cf47b-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/fc9ea964416c483bb832ad57810cf47b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 202 COLUMNS
At line 1555 RHS
At line 1753 BOUNDS
At line 1909 ENDATA
Problem MODEL has 197 rows, 155 columns and 1043 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 22 SOS with 154 members
Cgl0004I processed model has 85 rows, 154 columns (154 integer (154 of which binary)) and 553 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution fou

,ID,name,SwE,PdM,Biz,CS,group
10,11,Kate,0,0,1,0,A
18,19,Tate,1,0,0,0,A
20,21,Valentino,1,0,0,0,A
12,13,Marries,0,0,0,1,B
21,22,Warner,0,0,1,0,B
1,2,Bob,1,0,0,0,B
6,7,Gordon,1,0,0,0,C
8,9,Iris,1,0,0,0,C
9,10,Jamie,0,1,0,0,C
4,5,Eli,0,0,0,1,D


In [186]:
result3.sort_values('group').to_csv('./result_03.csv', index=False)
